In [ ]:
import sys
import boto3
from datetime import datetime

# Define parameters
QUERY_RESULTS_BUCKET = 's3://my-athena-bucket-dec2024/'
MY_DATABASE = 'de_proj_database2'
SOURCE_PARQUET_TABLE_NAME = 'jobdetails_jobs_with_details_parquet_tbl'
NEW_PROD_PARQUET_TABLE_NAME = 'jobdetails_jobs_with_details_parquet_tbl_PROD'
NEW_PROD_PARQUET_TABLE_S3_BUCKET = 's3://parquet-jobdetails-table-prod-1'

# Create a string with the current UTC datetime and replace special characters
DATETIME_NOW_INT_STR = str(datetime.utcnow()).replace('-', '_').replace(' ', '_').replace(':', '_').replace('.', '_')

# Initialize Athena client
client = boto3.client('athena')

# Start the query execution to create the new production table
queryStart = client.start_query_execution(
    QueryString=f"""
    CREATE TABLE {NEW_PROD_PARQUET_TABLE_NAME}_{DATETIME_NOW_INT_STR} WITH (
        external_location='{NEW_PROD_PARQUET_TABLE_S3_BUCKET}/{DATETIME_NOW_INT_STR}/',
        format='PARQUET',
        write_compression='SNAPPY',
        partitioned_by = ARRAY['yr_mo_partition']
    ) AS
    SELECT
        *
    FROM "{MY_DATABASE}"."{SOURCE_PARQUET_TABLE_NAME}"
    ;
    """,
    QueryExecutionContext={
        'Database': f'{MY_DATABASE}'
    },
    ResultConfiguration={'OutputLocation': f'{QUERY_RESULTS_BUCKET}'}
)

# List of possible query statuses
resp = ["FAILED", "SUCCEEDED", "CANCELLED"]

# Get the query execution response
response = client.get_query_execution(QueryExecutionId=queryStart["QueryExecutionId"])

# Wait until the query finishes
while response["QueryExecution"]["Status"]["State"] not in resp:
    response = client.get_query_execution(QueryExecutionId=queryStart["QueryExecutionId"])

# Handle failed queries
if response["QueryExecution"]["Status"]["State"] == 'FAILED':
    sys.exit(response["QueryExecution"]["Status"]["StateChangeReason"])

# Log success
print(f"Production table created successfully: {NEW_PROD_PARQUET_TABLE_NAME}_{DATETIME_NOW_INT_STR}")
